In [3]:
import pandas as pd
import os
from pathlib import Path
import spacy
from sentence_transformers import SentenceTransformer
from spacy.lang.en.stop_words import STOP_WORDS

spacy.cli.download("en_core_web_sm")

# Smaller BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])
nlp.add_pipe("sentencizer")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.3 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
DATA_DIR = Path.cwd() / "data"

DF_DIR = Path.cwd() / "saved_dfs"
DF_DIR.mkdir(exist_ok=True)

df_path = DF_DIR / "dataframe.csv"

# Conditional that checks whether we saved the dfs as csv files in prior run.
# If yes, then reinitialise these csvs as dfs.
# If not, then create the dfs and save them in csv format for next run.
if df_path.exists():
    print("Loading dataset from CSV...")
    df = pd.read_csv(df_path)

else:
    data = []

    for filename in os.listdir(DATA_DIR):
        if filename.endswith(".txt"):
            # Print filename for progress indication
            print(filename)

            file_path = DATA_DIR / filename
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
                # Use spaCy to process the content into sentences
                doc = nlp(content)
                sentences = [sent.text.strip() for sent in doc.sents]
                
                for sentence in sentences:
                    # Append each sentence and filename to a list
                    data.append({"filename": filename, "sentence": sentence})


    # Convert the list of sentence/filename dictionaries into a dataframe 
    df = pd.DataFrame(data)
    df.to_csv(df_path, index=False)

print(df.head())

Loading dataset from CSV...
                                            filename  \
0  Diarrhea-based_Detoxification_Hotel_By_Medical...   
1  Diarrhea-based_Detoxification_Hotel_By_Medical...   
2  Diarrhea-based_Detoxification_Hotel_By_Medical...   
3  Diarrhea-based_Detoxification_Hotel_By_Medical...   
4  Diarrhea-based_Detoxification_Hotel_By_Medical...   

                                            sentence  
0  A novel new 40-million-US dollars health clini...  
1  It\nwill be the world's first detoxification h...  
2                                     I kid you not.  
3  Dr Somchai Hongnamsakul of Johns Hopkins Unive...  
4  Explaining the new therapy, Dr Hongnamsakul sa...  


In [10]:
import joblib

version = 1

EMBEDDING_DIR = Path.cwd() / "embeddings"
EMBEDDING_DIR.mkdir(exist_ok=True)

embeddings_path = EMBEDDING_DIR / f'sentence_embeddings_v{version}.joblib'

# Conditional to check whether our embeddings joblib already exists from prior runs.
if not embeddings_path.exists():
    print("Generating embeddings for the dataset...")
    embeddings = model.encode(df['sentence'].tolist(), show_progress_bar=True)
    joblib.dump(embeddings, embeddings_path)
else:
    print("Loading embeddings from file...")
    embeddings = joblib.load(embeddings_path)

Loading embeddings from file...


In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(query, embeddings, df):
    # Encode the query string using the model to get its embedding.
    query_embedding = model.encode([query]) 

    # Calculate the cosine similarity between the query embedding and all embeddings in the dataset.
    # cosine_similarity returns a matrix where each row is the similarity of the query to each document.
    # We take the first row [0] because there's only one query, resulting in a one-dimensional array of similarities.
    similarities = cosine_similarity(query_embedding, embeddings)[0]

    top_indices = np.argsort(similarities)[-20:]

    # Use DataFrame.iloc to select the rows at the given indices (top_indices).
    # This gives us the rows from the dataframe that correspond to the top 20 similarities.
    top_docs = df.iloc[top_indices]

    # Select the corresponding top similarity scores using the indices.
    # This gives us the actual similarity scores of the top 20 matches.
    top_scores = similarities[top_indices]

    # Return the top matching documents and their similarity scores.
    return top_docs[['sentence', 'filename']], top_scores

In [12]:
test_set_columns=["Query", "Result", "Cosine", "Filename"]

def create_test_set(query, embeddings, df, test_set):
    top_docs, top_scores = search(query, embeddings, df)
    
    if top_docs.empty:
        print("No documents found for this query.")
        return test_set

    new_rows = []
    # Use zip to positionally combine and iterate over the df and scores in parallel. 
    # iterrows() is used to return a tuple of (index, Series) from the df.
    for (index, row), score in zip(top_docs.iterrows(), top_scores):
        new_row = {
            "Query": query,
            "Result": row['sentence'],
            "Cosine": score,
            "Filename": row['filename'],
        }
        new_rows.append(new_row)
    
    new_rows_df = pd.DataFrame(new_rows)
    test_set = pd.concat([test_set, new_rows_df], ignore_index=True)
    
    return test_set

In [13]:
def generate_test_set_from_queries(query_file_path, embeddings, df):
    test_set = pd.DataFrame(columns=test_set_columns)
    with open(query_file_path, 'r') as file:
        queries = file.read().splitlines()
    
    for query in queries:
        test_set = create_test_set(query, embeddings, df, test_set)
    
    return test_set

TEST_SET_DIR = Path.cwd() / "test_sets"
QUERIES_DIR = Path.cwd() / "queries"
TEST_SET_DIR.mkdir(exist_ok=True)
QUERIES_DIR.mkdir(exist_ok=True)
test_set_path = TEST_SET_DIR / f"test_set_v{version}.csv"
query_file_path = QUERIES_DIR / "queries.txt"

test_set = generate_test_set_from_queries(query_file_path, embeddings, df)

test_set.to_csv(test_set_path, index=False)

In [14]:
def normalize_sentence(sentence):
    # Need to remove any extra spaces/linebreaks
    # The original df sentences sometimes keep their raw formatting (weird line breaks)
    # This leads to a failure to match the embedding search results to my manually curated results
    sentence = sentence.replace("\n", " ")
    return sentence.strip()

def compute_evaluations(test_set, relevant_results):
    evaluation_data = []

    # My manually selected results (from keyword search)
    # are grouped by a set of possible analogous queries.
    # This loop iterates over each row that contains the same query group.
    # After each row in the matching column has been iterated over, it iterates over the rows for the next query group.
    for query_group in relevant_results['Query'].unique():
        grouped_queries = query_group.split(',')
        # Create a list of the normalised results from the relevant result dataframe (filtered on the query group)
        relevant_set = [normalize_sentence(sentence) for sentence in relevant_results[relevant_results['Query'] == query_group]['Result']]
        
        total_hits = 0
        matching_sentences = []

        # Iterate through the queries in the query group, e.g. ('What is arthritis', 'What is arthritis caused by')
        for query in grouped_queries:
            query = query.strip()
            # Find all the results from the test_set that pertain to the individual query (results from embedding search)
            query_results = test_set[test_set['Query'] == query]['Result'].apply(normalize_sentence)

            # Count the sentences that appear in both my manually collated relevant_results and the embedding search results
            for sentence in query_results:
                if sentence in relevant_set:
                    total_hits += 1
                    matching_sentences.append(sentence)

        evaluation_data.append({
            'Query Group': query_group,
            'Total Hits': total_hits,
            'Matching Sentences': ', '.join(matching_sentences)
        })
    
    eval_df = pd.DataFrame(evaluation_data)
    
    return eval_df

EVAL_DIR = Path.cwd() / "evaluations"
EVAL_DIR.mkdir(exist_ok=True)

relevant_results_path = QUERIES_DIR / "relevant_query_results.csv"
relevant_results = pd.read_csv(relevant_results_path)

test_set_path = TEST_SET_DIR / f"test_set_v{version}.csv"
test_set = pd.read_csv(test_set_path)

eval_df = compute_evaluations(test_set, relevant_results)
eval_path = EVAL_DIR / f"evaluation_v{version}.csv"
eval_df.to_csv(eval_path, index=False)

print(eval_df.head())

                                         Query Group  Total Hits  \
0              Is salt unhealthy, Salt damages cells           0   
1                     What are signs of intelligence           1   
2  What are signs of intelligence, What are signs...           0   
3                        How to gain weight quickly            0   
4     What is arthritis, What is arthritis caused by           2   

                                  Matching Sentences  
0                                                     
1  All hyperactive children have potential genius...  
2                                                     
3                                                     
4  The cooked saturated fat are a problem, but th...  
